# Planning System

> Aim: create a agent that takes all the data (puzzle, problem statement, etc) and the related solutions and formulates a plan (for the coding agent) on how to solve the puzzle.

In [ ]:
# Puzzle: 2024 day 4
PUZZLE = """
"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:


..X...
.SAMX.
.A..A.
XMAS.S
.X....
The actual word search will be full of letters instead. For example:

MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with .:

....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
Take a look at the little Elf's word search. How many times does XMAS appear?
"""

## Setup the notebook

In [2]:
import os
import sys
from dotenv import load_dotenv

# Append the models path in order to import the models
PROJECT_ROOT = os.path.join(os.path.abspath(""), 'src/')


print(PROJECT_ROOT)

sys.path.append(PROJECT_ROOT)

from models.gemini_model import GeminiLanguageModel
from agents.pre_processing_agent import PreProcessingAgent
from agents.retreival_agent import RetrievalAgent
from agents.base_agent import MockAgent, BaseAgent
from utils.util_types import AgentSettings, Puzzle
from core.orchestrator import Orchestrator
from core.state import MainState

/home/twanh/workspace/thesis/thesis-advent-of-agents/src/


In [ ]:
load_dotenv()

True

In [4]:
# Create a model that can be used
model = GeminiLanguageModel(
    api_key=os.getenv("GEMINI_API_KEY"),
    model_name='gemini-2.0-flash'
)

In [5]:
agents: tuple[tuple[BaseAgent, AgentSettings], ...] = (
    (
        PreProcessingAgent('preprocess', model=model),
        AgentSettings(enabled=True, can_debug=False),
    ),
    (
        RetrievalAgent(
            'retreival',
            model=model,
            connection_string=os.getenv('DB_CONNECTION_STRING') or '',
            openai_key=os.getenv('OPENAI_API_KEY') or '',
        ),
        AgentSettings(enabled=True, can_debug=False),
    ),
)


orchestrator = Orchestrator(agents, {})


puzzle = Puzzle(
    description=PUZZLE,
    solution=None,
    year=2024,
    day=5,
)

state = MainState(puzzle=puzzle)

return_state = orchestrator.solve_puzzle(state)

2025-05-07 11:20:28.260 | INFO     | core.retreival:init_db:154 - Database initialization complete.
2025-05-07 11:20:28.263 | INFO     | core.orchestrator:solve_puzzle:56 - Running agent: preprocess
2025-05-07 11:20:28.266 | DEBUG    | agents.pre_processing_agent:process:21 - Preprocessing agent prompt: You are a pre-processing agent. Your task is to process the following Advent of Code puzzle description so that the output can be stored in a RAG (Retrieval-Augmented Generation) database and later used by a planning agent. Follow these steps precisely:

------------------------------------------------------------
Step 1: Detailed Extraction
------------------------------------------------------------
Extract every technical detail from the puzzle and separate it from the narrative. Specifically, please extract:
  - Core Problem Statement:
      - A concise description of the computational task (exclude story or decorative language).
  - Input Specifications:
     - Detailed description

In [6]:
from pprint import pprint, pformat

pprint(return_state)

MainState(puzzle=Puzzle(description='\n"Looks like the Chief\'s not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!\n\nAs the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she\'d like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.\n\nThis word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It\'s a little unusual, though, as you don\'t merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:\n\n\n..X...\n.SAMX.\n.A..A.\nXMAS.S\n.X....\nThe actual word search will be full of letters instead. For example:\n\nMMMSXXMASM\nMSAMXMSMSA\nAMXSXMAAMM\nMSAMASMSMX\nXMASAMXAMM\nXXAMMXXAMA\nSMSMSASXSS\nSAXAMA

## Planning 1: Step by step plan

In [ ]:
PROMPT_STEP_BY_STEP = """
You are an AI agent specializing in creating plans on how to solve programming puzzles, specifically Advent of Code problems.
Your task is given an (advent of code) puzzle you will create a plan on how to solve the problem using code that will be passed on to a coder.
Follow these steps precisely:

Your input will be a JSON object containing the following:

{{
  "problem_statement": "The problem statement (string)",
  "full_description": "The full description of the problem (string)",
  "underlying_concepts": ["string"],
  "keywords": ["string"],
  "constraints": ["string"],
  "example_solutions": [
        {{
        "puzzle": "Previous puzzle title or brief description",
        "plan": "The plan on how to solve this previous puzzle",
        "code": "The code used to solve this previous puzzle (can be empty if not available)"
        }}
  ]
}}

Based on the provided input, generate a comprehensive, step-by-step plan for solving the current Advent of Code puzzle. Your plan should be clear, detailed, and directly usable by a coding agent to implement the solution without needing to figure out the high-level logic itself.

**Process:**

------------------------------------------------------------
Step 1: Understand the Problem:
------------------------------------------------------------
- Carefully read and analyze the `problem_statement`, `full_description`, `underlying_concepts`, `keywords`, and `constraints`.
- Identify the core objective of the puzzle, the precise input format, the required output format, and any specific rules, conditions, or limitations mentioned in the constraints or description.
- Formulate a clear internal understanding of the problem's requirements and what constitutes a correct solution.

------------------------------------------------------------
Step 2: Analyze Example Solutions (if provided):
------------------------------------------------------------
- If the `example_solutions` array contains entries, examine the `plan` and `code` provided for each previous puzzle.
- Identify common patterns, data structures, algorithms, or general problem-solving strategies that were successful in similar problems.
- Note how input was parsed, how logic was applied, how constraints were handled, and how the final output was generated in those examples.
- Evaluate which approaches or techniques from the examples might be relevant, adaptable, or directly applicable to the current puzzle. If no examples are provided or relevant, acknowledge this and proceed based solely on the current problem description.

------------------------------------------------------------
Step 3: Formulate the Detailed Plan:
------------------------------------------------------------

Using the deep understanding of the problem acquired in Step 1 and the relevant insights gained from analyzing example solutions (if provided) in Step 2, 
**construct a comprehensive, step-by-step implementation plan** for the current Advent of Code puzzle.

This plan is specifically intended for a coding agent and must be **actionable, unambiguous, and detailed enough** for them to translate directly into python code without requiring further 
high-level problem-solving.

Follow these instructions while formulating the plan:

1.  **Structure the Solution:** Break the overall solution process into a sequence of distinct, logical steps. Think of these as the major phases or components of the implementation.
2.  **Detail Each Step:** For *each* step you define in the plan, provide the following specific information:
    *   **Objective:** Clearly state the purpose of this step and what it aims to accomplish.
    *   **Input:** Specify what data or intermediate results this step receives or operates on.
    *   **Output:** Define the data structure or result this step produces for subsequent steps.
    *   **Techniques:** Note any specific algorithms, data structures, or core programming techniques that should be used within this step (e.g., "Use a hash map to count frequencies", "Sort this list", "Perform a depth-first search").
    *   **Constraints/Rules:** Mention how any relevant constraints, edge cases, or specific puzzle rules apply *to this particular step*.
3.  **Cover the Full Workflow:** Ensure the sequence of steps starts with processing the initial puzzle input (if necessary) and progresses through all required calculations, transformations, and logic up to the point where the final answer is determined.
4.  **Identify Core Components:** Explicitly name and describe the main data structures and significant algorithms that are central to the overall solution's logic, potentially across multiple steps.
5.  **Highlight Critical Considerations:** Call out any important performance implications, tricky edge cases, or specific constraints (like memory limits or time limits, if applicable) that require careful consideration during implementation.

------------------------------------------------------------
Step 4: Generate Structured Output Format
------------------------------------------------------------

Your final output must be *only* the structured plan, formatted using markdown. 
Do not include any conversational text, explanations about your process, or preamble before the plan. The plan should follow this structure:

```markdown

## 1. Overall Approach
A high-level description of the main strategy or algorithm chosen to solve the problem (e.g., "Iterate through the input applying rules...", "Use dynamic programming...", "Represent as a graph and find shortest path...", etc.).

## 2. Data Structures
List and describe the primary data structures that will be needed (e.g., `list` to store numbers, `dictionary` for mapping, `set` for unique items, a custom `Node` class, a 2D array for a grid). Explain briefly why each is suitable.

## 3. Detailed Steps
A numbered or bulleted list outlining the precise sequence of steps the coding agent should follow to implement the solution. Be specific and unambiguous.

*   **Step 3.1: Input Parsing**
    *   Description: How to read the input data.
    *   Input: Raw input string/lines.
    *   Output: Structured data (e.g., list of integers, grid, list of objects).
    *   Details: Specify format expectations and how to handle lines/sections.
*   **Step 3.2: [Next Logical Step]**
    *   Description: ...
    *   Input: ...
    *   Output: ...
    *   Details: ...
*   ... continue with all necessary steps ...
*   **Step X: Generate Output**
    *   Description: Format the final result according to the puzzle's output requirements.
    *   Input: The final calculated value(s) or structure(s).
    *   Output: The final string/number/formatted text.
    *   Details: Ensure it matches the exact format specified.

## 4. Algorithms/Techniques
List any specific algorithms or significant techniques used across multiple steps or as a core part of the solution (e.g., Sorting, Searching (BFS/DFS), Dynamic Programming, Recursion, Regex, Specific Mathematical Formulas).

## 5. Edge Cases and Constraints
List specific conditions or constraints from the problem description that require special handling (e.g., "Input numbers can be negative", "The grid might be empty", "Performance is critical for large inputs", "Handle invalid input format gracefully"). Remind the coder to be mindful of these.

## 6. Expected Final Output Format
Reiterate the exact format required for the final answer (e.g., "A single integer", "A string", "Two integers separated by a comma").

------------------------------------------------------------
Your input is:

{json_input}

"""

In [13]:
import json

retreived_to_dict = [
    {
        'puzzle': ret[0].description,
        'plan': ret[1],
        'code': ret[0].solution 
    } 
    for ret in return_state.retreived_puzzles
]

inp = {
  "problem_statement": return_state.problem_statement,
  "full_description": return_state.puzzle.description,
  "underlying_concepts": return_state.underlying_concepts,
  "keywords": return_state.keywords,
  "constraints": return_state.constraints,
  "example_solutions": [
        {
        "puzzle": "Previous puzzle title or brief description",
        "plan": "The plan on how to solve this previous puzzle",
        "code": "The code used to solve this previous puzzle (can be empty if not available)"
        }
  ]
}

json_inp = json.dumps(inp, indent=2)
json_inp

'{\n  "problem_statement": "Count the number of occurrences of the string \'XMAS\' in a grid of characters. The string can be found horizontally, vertically, diagonally, backwards, and overlapping other instances of the string.",\n  "full_description": "\\n\\"Looks like the Chief\'s not here. Next!\\" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!\\n\\nAs the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she\'d like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.\\n\\nThis word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It\'s a little unusual, though, as you don\'t merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have bee

In [14]:
formatted_prompt = PROMPT_STEP_BY_STEP.format(json_input=json_inp)
formatted_prompt

'\nYou are an AI agent specializing in creating plans on how to solve programming puzzles, specifically Advent of Code problems.\nYour task is given an (advent of code) puzzle you will create a plan on how to solve the problem using code that will be passed on to a coder.\nFollow these steps precisely:\n\nYour input will be a JSON object containing the following:\n\n{\n  "problem_statement": "The problem statement (string)",\n  "full_description": "The full description of the problem (string)",\n  "underlying_concepts": ["string"],\n  "keywords": ["string"],\n  "constraints": ["string"],\n  "example_solutions": [\n        {\n        "puzzle": "Previous puzzle title or brief description",\n        "plan": "The plan on how to solve this previous puzzle",\n        "code": "The code used to solve this previous puzzle (can be empty if not available)"\n        }\n  ]\n}\n\nBased on the provided input, generate a comprehensive, step-by-step plan for solving the current Advent of Code puzzle. 

In [15]:
ret = model.prompt(formatted_prompt)

2025-05-07 11:25:02.657 | DEBUG    | models.gemini_model:prompt:16 - Prompting GeminiLanguageModel(model_name='gemini-2.0-flash') with prompt: self.system_prompt=None text='\nYou are an AI agent specializing in creating plans on how to solve programming puzzles, specifically Advent of Code problems.\nYour task is given an (advent of code) puzzle you will create a plan on how to solve the problem using code that will be passed on to a coder.\nFollow these steps precisely:\n\nYour input will be a JSON object containing the following:\n\n{\n  "problem_statement": "The problem statement (string)",\n  "full_description": "The full description of the problem (string)",\n  "underlying_concepts": ["string"],\n  "keywords": ["string"],\n  "constraints": ["string"],\n  "example_solutions": [\n        {\n        "puzzle": "Previous puzzle title or brief description",\n        "plan": "The plan on how to solve this previous puzzle",\n        "code": "The code used to solve this previous puzzle (ca

In [16]:
print(ret)

```markdown
## 1. Overall Approach
The problem requires counting all occurrences of "XMAS" in a character grid, considering all eight directions (horizontal, vertical, diagonal, and their reverses). A brute-force approach is suitable, where we iterate through each cell of the grid and check for "XMAS" in all possible directions starting from that cell.

## 2. Data Structures
*   `grid`: A 2D list (list of lists) of characters representing the word search grid. This provides easy access to individual characters by row and column.
*   `xmas`: The target string "XMAS".

## 3. Detailed Steps
*   **Step 3.1: Input Parsing**
    *   Description: Read the input grid from the problem input string.
    *   Input: Raw input string (lines of characters).
    *   Output: `grid`: A 2D list of characters.
    *   Details: Split the input string into lines. Each line becomes a row in the `grid`. Each character in the line is an element in that row.

*   **Step 3.2: Initialize Count**
    *   Descript